In [16]:
from matplotlib import pyplot as plt
from matplotlib.dates import MonthLocator, num2date
from matplotlib.ticker import FuncFormatter
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import add_changepoints_to_plot

import pandas as pd
import numpy as np
import datetime as dt
from collections import defaultdict
import time
import datetime as dt
from pytz import timezone
tz = timezone('EST')
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()

In [17]:
nyc2010 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2010.csv")
nyc2011 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2011.csv")
nyc2012 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2012.csv")
nyc2013 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2013.csv")
nyc2014 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2014.csv")
nyc2015 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2015.csv")
nyc2016 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2016.csv")
nyc2017 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2017.csv")
nyc2018 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2018.csv")
nyc2019 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2019.csv")
nyc2020 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2020.csv")
nyc2021 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2021.csv")
nyc2022 = pd.read_csv(r"C:\Users\Rohan\Desktop\Smart Grids\electricity_load_data\2022.csv")

In [18]:
nyc2022

,RTD End Time Stamp,Zone Name,Zone PTID,RTD Actual Load
0,2022/01/01 00:05:00,N.Y.C.,61761,4623.9080
1,2022/01/01 00:10:00,N.Y.C.,61761,4642.4116
2,2022/01/01 00:15:00,N.Y.C.,61761,4606.2410
3,2022/01/01 00:20:00,N.Y.C.,61761,4589.0850
4,2022/01/01 00:25:00,N.Y.C.,61761,4574.2725
...,...,...,...,...
31430,2022/04/16 23:40:00,N.Y.C.,61761,4367.3170
31431,2022/04/16 23:45:00,N.Y.C.,61761,4341.1190
31432,2022/04/16 23:50:00,N.Y.C.,61761,4331.1170
31433,2022/04/16 23:55:00,N.Y.C.,61761,4286.6304


In [19]:
nyc_load = nyc2010
nyc_load = nyc_load.append(nyc2011, ignore_index = True)
nyc_load = nyc_load.append(nyc2012, ignore_index = True)
nyc_load = nyc_load.append(nyc2013, ignore_index = True)
nyc_load = nyc_load.append(nyc2014, ignore_index = True)
nyc_load = nyc_load.append(nyc2015, ignore_index = True)
nyc_load = nyc_load.append(nyc2016, ignore_index = True)
nyc_load = nyc_load.append(nyc2017, ignore_index = True)
nyc_load = nyc_load.append(nyc2018, ignore_index = True)
nyc_load = nyc_load.append(nyc2019, ignore_index = True)
nyc_load = nyc_load.append(nyc2020, ignore_index = True)
nyc_load = nyc_load.append(nyc2021, ignore_index = True)
nyc_load = nyc_load.append(nyc2022, ignore_index = True)

In [20]:
nyc_load

,RTD End Time Stamp,Zone Name,Zone PTID,RTD Actual Load
0,2010/01/01 00:05:00,N.Y.C.,61761,5493.5000
1,2010/01/01 00:10:00,N.Y.C.,61761,5460.0000
2,2010/01/01 00:15:00,N.Y.C.,61761,5457.8000
3,2010/01/01 00:20:00,N.Y.C.,61761,5413.7000
4,2010/01/01 00:25:00,N.Y.C.,61761,5384.6000
...,...,...,...,...
1308396,2022/04/16 23:40:00,N.Y.C.,61761,4367.3170
1308397,2022/04/16 23:45:00,N.Y.C.,61761,4341.1190
1308398,2022/04/16 23:50:00,N.Y.C.,61761,4331.1170
1308399,2022/04/16 23:55:00,N.Y.C.,61761,4286.6304


In [29]:
nyc_train = nyc_load[["RTD End Time Stamp", "RTD Actual Load"]]
nyc_train.rename(columns={"RTD End Time Stamp": "ds", "RTD Actual Load": "y"}, inplace=True)

C:\Users\Rohan\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [30]:
nyc_train

,ds,y
0,2010/01/01 00:05:00,5493.5000
1,2010/01/01 00:10:00,5460.0000
2,2010/01/01 00:15:00,5457.8000
3,2010/01/01 00:20:00,5413.7000
4,2010/01/01 00:25:00,5384.6000
...,...,...
1308396,2022/04/16 23:40:00,4367.3170
1308397,2022/04/16 23:45:00,4341.1190
1308398,2022/04/16 23:50:00,4331.1170
1308399,2022/04/16 23:55:00,4286.6304


In [31]:
model = Prophet(
        changepoint_prior_scale=0.5, 
        seasonality_mode='multiplicative', 
        interval_width=0.95, 
    )
model.add_country_holidays(country_name='US')
model.fit(nyc_train)

In [32]:
model.fit(nyc_train)

In [34]:
import json
from prophet.serialize import model_to_json, model_from_json

with open('nyc_model.json', 'w') as fout:
    json.dump(model_to_json(model), fout)  # Save model

#with open('serialized_model.json', 'r') as fin:
    #m = model_from_json(json.load(fin))  # Load model